In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.trainer import TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_int8_training, PeftModel
from trl import SFTTrainer, DPOTrainer
from datasets import load_dataset, Dataset
from transformers import pipeline
import numpy as np
import json

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preparing the Glan Prompting Data in preference pairs

In [ ]:
glan_prompt_answers = "../data/glan_prompt_data/a/answers.json"
glan_prompt_questions = "../data/glan_prompt_data/q/questions.json"

In [ ]:
with open(glan_prompt_questions, 'rb') as file:
    questions = json.load(file)
    
with open(glan_prompt_answers, 'rb') as file:
    answers = json.load(file)

In [ ]:
question_list = []
preferred_answer = []
rejected_answer = []

for key in questions.keys():
    question_list.append(questions[key]['q'])
    preferred_answer.append(answers[key][0]['better'])
    rejected_answer.append(answers[key][0]['worse'])

glan_data_pairs = {   
                        'question': question_list,
                        'preferred_answer': preferred_answer,
                        'rejected_answer': rejected_answer
                    }

glan_dataset = Dataset.from_dict(glan_data_pairs)

### Initializing the tokenizer and prepare the data in chat template

In [ ]:
with open('hf_token.key', 'r') as f:
    hf_token = f.read()

base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-glan-sft"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
# Adding a special token for pad token so that eos token can be recognized 
# (https://github.com/unslothai/unsloth/issues/416)
# https://github.com/huggingface/transformers/issues/22794
# https://github.com/huggingface/transformers/issues/23230
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
tokenizer.padding_side = "right"
tokenizer.model_max_length = 512

In [ ]:
def format_chat_template(row):
    row_json = [
        {"role" : "user", "content": row['question']},
        {"role" : "assistant", "content": row['preferred_answer']}
    ]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [ ]:
dataset = glan_dataset.map(
                        format_chat_template,
                        num_proc=8
                    )

dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
# QLoRA Config for 4-bit quntization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# # For 8 bit quantization
# bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=200.0)

In [ ]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

model.config.use_cache=False
model.config.pad_token_id = tokenizer.pad_token_id # Updating the model config to use the special pad token

In [ ]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type = "CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    bf16=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=0.1,
    learning_rate=2e-4,
    logging_steps=5,
    logging_strategy="steps",
    log_level="info",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    group_by_length=True,
    report_to="none",
    seed=42
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
    packing= False
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

### Merging the base model with the adapter to get full model

In [ ]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model = "llama-3-8b-glan-sft"

In [ ]:
# Load Model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,    
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
    device_map={"":torch.cuda.current_device()}
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, padding='max_length', truncation=True, token = hf_token)
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
tokenizer.padding_side = "right"
tokenizer.model_max_length = 512

Merge adapter with the base model

In [ ]:
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [ ]:
model = model.merge_and_unload()
model.config.pad_token_id = tokenizer.pad_token_id # Updating the model config to use the special pad token

In [ ]:
model.save_pretrained("llama-3-8b-glan-sft-merged")
tokenizer.save_pretrained("llama-3-8b-glan-sft-merged")

### Load merged Model and Tokenizer for Inference

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "llama-3-8b-glan-sft-merged",
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.65s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained("llama-3-8b-glan-sft-merged")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

In [7]:
model.config.use_cache = True

messages = [
    {
        "role": "user",
        "content": "How do I overcome emotional trauma after breaking up with my partner?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

generation_config = model.generation_config
generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.repetition_penalty = 1.5

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1,
    generation_config=generation_config
)

text = tokenizer.decode(outputs[0], skip_special_tokens=False)

print(text.split("assistant")[1])

<|end_header_id|>

Breaking up from a relationship can be incredibly painful and lead to significant emotional trauma. It's important to allow yourself time to grieve the loss of the partnership while also focusing on self-care activities that promote healing such as:

1. Seeking support: Reach out to trusted friends, family members or join a support group for people going through similar experiences.
2. Processing your emotions: Write in a journal, engage in expressive arts therapy like painting or writing poetry about how you feel without judgment.

3.Celebrate past memories positively rather than dwelling negative aspects of ex-partnership which may trigger further pain
4.Set boundaries around social media use related to former partners 
5.Focus now more forward by setting goals and working towards personal growth & development 

Remember it takes different amount times heal process so patience compassion understanding is key during this journey back independence empowerment You des

### Direct Preference Optimization

In [ ]:
base_model = "llama-3-8b-glan-sft-merged"
new_model = "llama-3-8b-glan-dpo"

In [ ]:
# Tokenizer from pre-trained model
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
# QLoRA Config for 4-bit quntization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# # For 8 bit quantization
# bnb_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=200.0)

In [ ]:
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

model.config.use_cache=False
# model.config.pad_token_id = tokenizer.pad_token_id # Updating the model config to use the special pad token

In [ ]:
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type = "CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [ ]:
def format_chat_template_dpo(row):
    row_json = [
        {"role" : "user", "content": row['question']}
    ]

    prompt = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)
    chosen = str(row['preferred_answer']) + tokenizer.eos_token
    rejected = str(row['rejected_answer']) + tokenizer.eos_token

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected
    }

In [ ]:
def get_counsel_chat_paired(sanity_check=False, cache_dir=None, num_proc=24):

    original_columns=glan_dataset.column_names
    
    dataset = glan_dataset.map(
                        format_chat_template_dpo,
                        num_proc=24,
                        remove_columns=original_columns
                    )

    dataset = dataset.train_test_split(test_size=0.1)
    
    return dataset

In [ ]:
dataset = get_counsel_chat_paired()

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    bf16=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=0.1,
    learning_rate=2e-4,
    logging_steps=5,
    logging_strategy="steps",
    log_level="info",
    save_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    remove_unused_columns=False,
    warmup_steps=10,
    report_to="none",
    seed=42
)

In [ ]:
trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_arguments,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_prompt_length=1024,
    max_length=1024
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

### Merging the base model with the adapter to get full model

In [ ]:
base_model = "llama-3-8b-glan-sft-merged"
new_model = "llama-3-8b-glan-dpo"

In [ ]:
# Load Model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
    device_map={"":torch.cuda.current_device()}
)

In [ ]:
# Tokenizer from pre-trained model
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
model = PeftModel.from_pretrained(base_model_reload, new_model)
model = model.merge_and_unload()

In [ ]:
model.save_pretrained("llama-3-8b-glan-dpo-merged")
tokenizer.save_pretrained("llama-3-8b-glan-dpo-merged")

### Loading the merged model for inference

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "llama-3-8b-glan-dpo-merged",
    torch_dtype = torch.bfloat16,
    device_map={'':torch.cuda.current_device()}
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


In [9]:
tokenizer = AutoTokenizer.from_pretrained("llama-3-8b-glan-dpo-merged")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

In [11]:
model.config.use_cache = True

messages = [
    {
        "role": "user",
        "content": "How do I overcome emotional trauma after breaking up with my partner?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

generation_config = model.generation_config
generation_config.pad_token_id = tokenizer.pad_token_id
# generation_config.repetition_penalty = 1.5

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1,
    generation_config=generation_config
)

text = tokenizer.decode(outputs[0], skip_special_tokens=False)

print(text.split("assistant")[1])

<|end_header_id|>

Overcoming emotional trauma after a breakup can be a challenging and painful process, but it is absolutely possible with the right support and guidance. Some strategies that may help you heal include:

1. Allow yourself to grieve the loss: It's important to acknowledge and process the feelings of loss and grief that come with the end of a relationship.
2. Seek a safe space: Surround yourself with a supportive network of friends and family, or consider seeking therapy to provide a safe space for your emotions.
3. Practice self-care: Take care of your physical and emotional needs by ensuring you get enough sleep, eating healthily, and engaging in regular exercise.
4. Process your emotions: Write in a journal, create art, or engage in any other activity that helps you express and make sense of your emotions.
5. Give yourself time: Healing from emotional trauma takes time, so be patient and compassionate with yourself throughout the journey.
6. Consider therapy: A therap